This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
import requests
import json
from datetime import date

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned


def specify_nasdaq_api_url(exchange,code,date1,date2,api_key):
    url = f'https://data.nasdaq.com/api/v3/datasets/{exchange}/{code}.json?start_date={date1}&end_date={date2}&order=asc&api_key={api_key}'
    return url

def request_data(url):
    return requests.get(url)

In [4]:
res = request_data(specify_nasdaq_api_url('FSE','AFX_X', '2020-11-30', '2020-12-01',API_KEY))

In [5]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(json.dumps(res.json(), indent=4))

{
    "dataset": {
        "id": 10095370,
        "dataset_code": "AFX_X",
        "database_code": "FSE",
        "name": "Carl Zeiss Meditec (AFX_X)",
        "description": "Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704",
        "refreshed_at": "2020-12-01T14:48:09.907Z",
        "newest_available_date": "2020-12-01",
        "oldest_available_date": "2000-06-07",
        "column_names": [
            "Date",
            "Open",
            "High",
            "Low",
            "Close",
            "Change",
            "Traded Volume",
            "Turnover",
            "Last Price of the Day",
            "Daily Traded Units",
            "Daily Turnover"
        ],
        "frequency": "daily",
        "type": "Time Series",
        "premium": false,
        "limit": null,
        "transform": null,
        "column_index": null,
        "start_date": "2020-11-30",
        "end_date": 

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

    #1&2

In [6]:
afx_x_2017 = request_data(specify_nasdaq_api_url('FSE','AFX_X', '2016-12-31', '2017-12-31',API_KEY)) #1
afx_x_2017_dict = afx_x_2017.json() #2

In [7]:
dates = [entry[0] for entry in afx_x_2017_dict['dataset']['data']]

opens = [entry[1] for entry in afx_x_2017_dict['dataset']['data']]
nones = [idx for idx, val in enumerate(opens) if val == None]

for i in nones[::-1]:
    opens.pop(i)
    dates.pop(i)

In [8]:
nones

[74, 75, 85]

In [10]:
def get_max_min_open(data, dates = dates):
    
    max_val = max(data)
    min_val = min(data)
    
    idxmax = data.index(max_val)
    idxmin = data.index(min_val)
    
    strmax = f'Highest open price of ${max_val} occured on {dates[idxmax]}'
    strmin = f'Lowest open price of ${min_val} occured on {dates[idxmin]}'
    print(strmax,'\n', strmin)

    #3

In [11]:
get_max_min_open(opens)

Highest open price of $53.11 occured on 2017-12-14 
 Lowest open price of $34.0 occured on 2017-01-24


    #4

In [12]:
diffs = [entry[2] - entry[3] for entry in afx_x_2017_dict['dataset']['data']]
print(max(diffs))

2.8100000000000023


    #5

In [13]:
closing = [entry[4] for entry in afx_x_2017_dict['dataset']['data']]
closing_shifted = closing[1:]
closing_shifted.extend([0])
closing_diff = [closing_shifted[i] - closing[i] for i,v in enumerate(closing)]

    The greatest increase between days at closing:

In [14]:
max(closing_diff[:-1]) # skip the last value since it is shifted to an unknown value.

1.7199999999999989

    The greatest decrease between days at closing:

In [15]:
min(closing_diff[:-1])

-2.559999999999995

    #6

In [16]:
volume = [entry[6] for entry in afx_x_2017_dict['dataset']['data']]

In [17]:
sum(volume)/len(volume)

89124.33725490196

    #7

In [18]:
def med_odd(vals):
    sort_vals = sorted(vals)
    mid_val = sort_vals[len(sort_vals) // 2] # get the upper middle value
    return mid_val

def med_even(vals):
    sort_vals = sorted(vals)
    mid_val = len(sort_vals) // 2
    return (sort_vals[mid_val] + sort_vals[mid_val - 1])/2

def median(vals):
    return med_even(vals) if  len(vals) %2 ==0 else med_odd(vals)

In [19]:
median(volume)

76286.0